# HDFS

HDFS is het distributed file system van Hadoop dat de basis vormt voor een breed gamma van applicaties, waaronder MapReduce.
Dit framework maakt het mogelijk om niet-gespecialiseerde hardware te gebruiken om eenvoudig datacenters op te zetten of rekenclusters te beheren.
HDFS bereikt dit doel op een aantal manieren:
* Ten eerste wordt een file opgedeeld in verschillende blokken. Deze worden verdeeld over verschillende computers zodat code meerdere delen van het bestand kan gebruiken in parallel om zo de benodigde rekenkracht te verdelen over meerdere toestellen.
* Daarnaast worden de blokken ook gedupliceerd om zo fout-toleranter te zijn in het geval van een crash (hardware of software), stroomstoring, netwerkonderbreking.

Om dit te bereiken gebruikt Hadoop een Master-Slave architectuur dat bestaat uit een enkele namenode (master) en meerdere datanodes (slaves).
De namenode houdt bij hoeveel datanodes er actief zijn, welke blokken ze hebben en welke blokken bij welke file horen.
Indien er een datanode crasht gaat deze server dit detecteren en dit oplossen door de nodige blokken te kopieren van een andere datanode zodat er steeds voldoende kopies in het systeem aanwezig zijn.
Bij elke actie die uitgevoerd moet worden in het HDFS moet er steeds gevraagd worden aan de namenode welke blokken op welke datanodes we nodig hebben voor de gewenste file uit te lezen of code voor uit te voeren.
Het is dus duidelijk dat deze namenode een single-point-of-failure is wat ideaal is voor de availability van de cluster.
Dit kan opgelost worden door HDFS te runnen in een high-availability mode wat ervoor zorgt dat er steeds een backup aanwezig is voor de namenode die zeer snel de werking kan overnemen.
Deze structuur maakt het eenvoudig om aan horizontal scaling te doen door extra servers toe te voegen zonder dat er downtime is voor de hele cluster.

Dit resulteer in de volgende kenmerken van HDFS:
* High Throughput
* Scalability
* High Availability
* Data Reliability
* Fault Tolerance

## Starten en stoppen van de cluster

De cluster kan gestart worden door de volgende commando's uit te voeren in de commandline:
* start-dfs.sh
* start-yarn.sh

Eenmaal dat de cluster gestart is kunnen de lopende java applicaties gecontroleerd worden door middel van het commando jps.
Dit kan ook uitgevoerd worden vanuit deze jupyter notebook door onderstaande code-cell uit te voeren.

In [ ]:
!start-dfs.sh
!start-yarn.sh
!jps

Ook heeft de namenode een website waar naartoe kan gegaan worden om meer informatie over de cluster te bekijken. De standaardurl is [http://localhost:9870](http://localhost:9870).

Om de cluster terug te stoppen moeten in de terminal of de notebook de volgende commando's uitgevoerd worden.
* stop-dfs.sh
* stop-yarn.sh


In [ ]:
#!stop-yarn.sh
#!stop-dfs.sh

## Communiceren met het HDFS

Er zijn verschillende manieren om dit distributed bestandssysteem te gebruiken.
Ten eerste kan je gebruik maken van een command line interface (CLI) om bestanden uit te lezen, op te laden, ...
Daarnaast zijn er ook wrappers voor verschillende talen die toelaten om rechtstreeks vanuit code te interageren met het HDFS.
De voordelen van deze wrappers over een CLI zijn:
* Flexibiler om te interageren in applicaties.
* Gebruiksvriendelijker en gemakkelijker om te automatiseren.
* Eenvoudiger in onderhoud en te debuggen
* Volledige functionaliteit van een programmeertaal kan gebruikt worden zoals OO.

### Instantiering van een client

Veel verschillende talen beschikken over een wrapper om te communiceren met een HDFS.
In deze cursus gaan we gebruik maken van [de pydoop](https://crs4.github.io/pydoop/index.html) wrapper.
De eerste stap in het gebruik van de wrapper is te bepalen hoe de namenode van het hdfs gevonden kan worden.
Dit gebeurt als volgt:

In [6]:
import pydoop.hdfs as hdfs
# local filesystem
localFS = hdfs.hdfs(host='')
# initialisation where host (name or ipadress) and port can be defined
client = hdfs.hdfs(host='localhost', port=9000)
# use hdfs defined in local hadoop configuration files
#client = hdfs.hdfs()

client.capacity()

31040933888

### Aanmaken van files en directories

Om nu bestanden en folders aan te maken op dit distributed file systeem kunnen onderstaande functies gebruikt worden.

In [7]:
#mkdir commando
# exists: hdfs dfs -test -d hdfs_path
# create: hdfs dfs -mkdir -p /bigdata/03_HDFS
if not client.exists('03_HDFS'):
    client.create_directory('03_HDFS')
# set working directories to reduce overhead of typing paths
client.set_working_directory('03_HDFS')
print(client.working_directory())

hdfs://localhost:9000/user/bigdata/03_HDFS/03_HDFS/03_HDFS/03_HDFS
/home/bigdata/DataspellProjects/Leerstof/03_HDFS/~/DataspellProjects/Leerstof/03_HDFS/~/DataspellProjects/Leerstof/03_HDFS


In [ ]:
# hadoop fs -put /path/in/linux /hdfs/path
# hadoop fs -get /hdfs/path /path/in/linux
localFS.copy("davinci_notebooks.txt", client, "davinci.txt")
client.copy("davinci_notebooks.txt", client, "davinci2.txt")
localFS.move("ulysses.txt", client, "ulysses.txt")

### Bekijken van het filesysteem

In [ ]:
# hdfs dfs -ls command
client.list_directory("")
client.list_directory("..")
client.used()
client.working_directory()
client.default_block_size()

# moet gecombineerd worden met head om het uitlezen te stoppen
# in tegenstelling tot pydoop waar files kunnen aangepast worden gaat dit niet via CLI
# hdfs dfs -cat <hdfs_filename> | head -n 3
f = client.open_file("ulysses.txt")
for i in range(5):
    print(f.readline())

### Aanpassen van het filesysteem

In [ ]:
# hadoop fs -mv oldname newname
client.rename("davinci2.txt", "davinci3.txt")
# hdfs dfs -rm path
# add -r tag for deleting directory
client.delete("davinci3.txt")
# hdfs dfs –setrep –w 3 /tmp/logs/file.txt
client.set_replication("davinci.txt", 5)

### Oefening

Schrijf python code dat controleert of een directory bestaat in het hdfs.
Indien nee wordt de directory aangemaakt.
Indien ja, worden alle files in de directory verwijderd om van een lege directory te starten.
Upload daarna een tekst-bestand naar keuze.

In [ ]:
if not client.exists('/user/bigdata/03_HDFS'):
    client.create_directory('/user/bigdata/03_HDFS')
else:
    # do some cleaning in case anything else than input.txt is present
    for f in client.list_directory("."):
        if not f["name"].endswith("input.txt"):
            client.delete(f["name"], True)
            
localFS.copy("davinci_notebooks.txt", client, "03_HDFS/davinci.txt")